In [1]:
import azureml.core
import pandas as pd
import numpy as np
import logging

from azureml.core import Workspace, Experiment, Dataset
from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget
from azureml.train.automl.run import AutoMLRun

#Connect to wrokspace and Experiment again
ws = Workspace('8c96f3ed-e53f-4400-a002-420605371b18','Azure_Project','FRT_Project')
experiment_name = 'automl_loan_approval_prediction'
experiment =  Experiment(ws, experiment_name)

#Connect to cpu cluster again
amlcompute_cluster_name = "cpu-cluster"
cts=ws.compute_targets
compute_target = cts[amlcompute_cluster_name]

#Connect to dataset from within te datastore again
datastore = ws.get_default_datastore() 
dataset = Dataset.Tabular.from_delimited_files(path=[(datastore,'dataset/Prepared_train.csv')])
dataset.to_pandas_dataframe().reset_index(drop=True)

#Connect to most recent run
run_id = "AutoML_9301b394-5cc4-49a7-beee-c3f879f52408"
remote_run = AutoMLRun(experiment,run_id)

test_dataset = Dataset.Tabular.from_delimited_files(path=[(datastore,'dataset/Prepared_train.csv')])
test_dataset.to_pandas_dataframe().reset_index(drop=True)



,Loan Amount,Funded Amount,Funded Amount Investor,Term,Batch Enrolled,Interest Rate,Grade,Sub Grade,Employment Duration,Home Ownership,...,Total Received Late Fee,Recoveries,Collection Recovery Fee,Collection 12 months Medical,Application Type,Last week Pay,Total Collection Amount,Total Current Balance,Total Revolving Credit Limit,Loan Status
0,10000,32236,12329.36286,59,2522922,11.135007,2,34,0,176346.62670,...,0.102055,2.498291,0.793724,0,0,49,31,311301,6619,0
1,3609,11940,12191.99692,59,1586599,12.237563,3,43,1,39833.92100,...,0.036181,2.377215,0.974821,0,0,109,53,182610,20885,0
2,28276,9311,21603.22455,59,2136391,12.545884,6,44,0,91506.69105,...,18.778660,4.316277,1.020075,0,0,66,34,89801,26155,0
3,11170,6954,17877.15585,59,2428731,16.731201,3,33,0,108286.57590,...,0.044131,0.107020,0.749971,0,0,39,40,9189,60214,0
4,16890,13226,13539.92667,59,5341619,15.008300,3,44,0,44234.82545,...,19.306646,1294.818751,0.368953,0,0,18,430,126029,22579,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67458,13601,6848,13175.28583,59,3193689,9.408858,3,14,0,83961.15003,...,0.023478,564.614852,0.865230,0,0,69,48,181775,34301,1
67459,8323,11046,15637.46301,59,1780517,9.972104,3,23,1,65491.12817,...,0.027095,2.015494,1.403368,0,0,14,37,22692,8714,0
67460,15897,32921,12329.45775,59,1761981,19.650943,1,61,0,34813.96985,...,0.028212,5.673092,1.607093,0,0,137,17,176857,42330,0
67461,16567,4975,21353.68465,59,2333412,13.169095,4,53,2,96938.83564,...,0.074508,1.157454,0.207608,0,0,73,61,361339,39075,0


In [2]:
remote_run.get_details()

{'runId': 'AutoML_9301b394-5cc4-49a7-beee-c3f879f52408',
 'target': 'local',
 'status': 'Completed',
 'startTimeUtc': '2022-02-02T17:27:39.507389Z',
 'endTimeUtc': '2022-02-02T18:39:29.690111Z',
 'services': {},
 'warnings': [{'source': 'JasmineService',
   'message': 'No scores improved over last 20 iterations, so experiment stopped early. This early stopping behavior can be disabled by setting enable_early_stopping = False in AutoMLConfig for notebook/python SDK runs.'}],
 'properties': {'num_iterations': '1000',
  'training_type': 'TrainFull',
  'acquisition_function': 'EI',
  'primary_metric': 'AUC_weighted',
  'train_split': '0',
  'acquisition_parameter': '0',
  'num_cross_validation': '3',
  'target': 'local',
  'AMLSettingsJsonString': '{"path":null,"name":"automl_loan_approval_prediction","subscription_id":"8c96f3ed-e53f-4400-a002-420605371b18","resource_group":"Azure_Project","workspace_name":"FRT_Project","region":"westus","compute_target":"local","spark_service":null,"azure

In [3]:
best_run , fitted_model = remote_run.get_output()
fitted_model.steps

[('datatransformer',
  DataTransformer(
      task='classification',
      is_onnx_compatible=False,
      enable_feature_sweeping=True,
      enable_dnn=False,
      force_text_dnn=False,
      feature_sweeping_timeout=86400,
      featurization_config=None,
      is_cross_validation=True,
      feature_sweeping_config={}
  )),
 ('MaxAbsScaler', MaxAbsScaler(copy=True)),
 ('LightGBMClassifier',
  LightGBMClassifier(
      boosting_type='gbdt',
      colsample_bytree=0.7922222222222222,
      learning_rate=0.04737368421052632,
      max_bin=380,
      max_depth=4,
      min_child_weight=8,
      min_data_in_leaf=0.03793724137931035,
      min_split_gain=0.3684210526315789,
      n_estimators=400,
      num_leaves=224,
      reg_alpha=0,
      reg_lambda=0.6842105263157894,
      subsample=0.14894736842105263,
      random_state=None,
      n_jobs=-1,
      problem_info=ProblemInfo(
          gpu_training_param_dict={'processing_unit_type': 'cpu'}
      )
  ))]

In [4]:
fitted_model.named_steps['datatransformer'].get_engineered_feature_names()

['Application Type_ModeCatImputer_LabelEncoder',
 'Batch Enrolled_CharGramCountVectorizer_1104812',
 'Batch Enrolled_CharGramCountVectorizer_1135695',
 'Batch Enrolled_CharGramCountVectorizer_1184694',
 'Batch Enrolled_CharGramCountVectorizer_1467036',
 'Batch Enrolled_CharGramCountVectorizer_1586599',
 'Batch Enrolled_CharGramCountVectorizer_1761981',
 'Batch Enrolled_CharGramCountVectorizer_1766061',
 'Batch Enrolled_CharGramCountVectorizer_1780517',
 'Batch Enrolled_CharGramCountVectorizer_1930365',
 'Batch Enrolled_CharGramCountVectorizer_2003848',
 'Batch Enrolled_CharGramCountVectorizer_2078974',
 'Batch Enrolled_CharGramCountVectorizer_2136391',
 'Batch Enrolled_CharGramCountVectorizer_224923',
 'Batch Enrolled_CharGramCountVectorizer_2252229',
 'Batch Enrolled_CharGramCountVectorizer_2333412',
 'Batch Enrolled_CharGramCountVectorizer_2428731',
 'Batch Enrolled_CharGramCountVectorizer_2522922',
 'Batch Enrolled_CharGramCountVectorizer_2558388',
 'Batch Enrolled_CharGramCountVect

In [5]:
featurization_summay = fitted_model.named_steps['datatransformer'].get_featurization_summary()
pd.DataFrame.from_records(featurization_summay)

,RawFeatureName,TypeDetected,Dropped,EngineeredFeatureCount,Transformations
0,Application Type,Categorical,No,1,[ModeCatImputer-StringCast-LabelEncoder]
1,Batch Enrolled,Categorical,No,41,[StringCast-CharGramCountVectorizer]
2,Collection 12 months Medical,Categorical,No,1,[ModeCatImputer-StringCast-LabelEncoder]
3,Delinquency - two years,Categorical,No,9,[StringCast-CharGramCountVectorizer]
4,Employment Duration,Categorical,No,3,[StringCast-CharGramCountVectorizer]
5,Grade,Categorical,No,7,[StringCast-CharGramCountVectorizer]
6,Initial List Status,Categorical,No,1,[ModeCatImputer-StringCast-LabelEncoder]
7,Inquires - six months,Categorical,No,6,[StringCast-CharGramCountVectorizer]
8,Open Account,Categorical,No,36,[StringCast-CharGramCountVectorizer]
9,Public Record,Categorical,No,5,[StringCast-CharGramCountVectorizer]


In [32]:
run_id = "AutoML_9301b394-5cc4-49a7-beee-c3f879f52408"
training_run = AutoMLRun(experiment,run_id)
training_run

Experiment,Id,Type,Status,Details Page,Docs Page
automl_loan_approval_prediction,AutoML_9301b394-5cc4-49a7-beee-c3f879f52408,automl,Completed,Link to Azure Machine Learning studio,Link to Documentation


In [21]:
best_run

Experiment,Id,Type,Status,Details Page,Docs Page
automl_loan_approval_prediction,AutoML_9301b394-5cc4-49a7-beee-c3f879f52408_16,,Completed,Link to Azure Machine Learning studio,Link to Documentation


In [33]:
model_name = best_run.properties["model_name"]
model_name

'AutoML9301b394516'

In [34]:
script_file_name = "inference/score.py"
best_run.download_file("outputs/scoring_file_v_1_0_0.py", "inference/score.py")

In [35]:
description = "AutoML Model trained on bank data to predict whether to give out loan or not to a perticular person or entity."
model = remote_run.register_model(model_name=model_name, 
                                  description=description, 
                                  tags=None)
print(remote_run.model_id)

AutoML9301b394516


In [36]:
from azureml.core import Model

print('Name:', model.name)
print('Version:', model.version)

Name: AutoML9301b394516
Version: 18


In [61]:
from azureml.core.compute import ComputeTarget
from azureml.core.compute import AksCompute
from azureml.core.compute_target import ComputeTargetException

# Choose a name for your AKS cluster
aks_name = 'automl-aks'

# Verify that cluster does not exist already
try:
    aks_target = ComputeTarget(workspace=ws, name=aks_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    # Use the default configuration (can also provide parameters to customize)
    prov_config = prov_config = AksCompute.provisioning_configuration(vm_size = "Standard_D2_v2",
                                                                      location = "Central US")

    # Create the cluster
    aks_target = ComputeTarget.create(workspace = ws, 
                                    name = aks_name, 
                                    provisioning_configuration = prov_config)

if aks_target.get_status() != "Succeeded":
    aks_target.wait_for_completion(show_output=True)

Found existing cluster, use it.


In [64]:
aks_target

AksCompute(workspace=Workspace.create(name='FRT_Project', subscription_id='8c96f3ed-e53f-4400-a002-420605371b18', resource_group='Azure_Project'), name=automl-aks, id=/subscriptions/8c96f3ed-e53f-4400-a002-420605371b18/resourceGroups/Azure_Project/providers/Microsoft.MachineLearningServices/workspaces/FRT_Project/computes/automl-aks, type=AKS, provisioning_state=Succeeded, location=westus, tags={})

In [66]:
# aks_target.wait_for_completion(show_output = True)
print(aks_target.provisioning_state)
print(aks_target.provisioning_errors)

Succeeded
None


In [67]:
from azureml.core.webservice import Webservice, AksWebservice
# Set the web service configuration (using default here)
aks_config = AksWebservice.deploy_configuration()

In [69]:
from azureml.core.model import InferenceConfig
from azureml.core.environment import Environment
from azureml.automl.core.shared import constants
best_run.download_file(constants.CONDA_ENV_FILE_PATH, 'myenv.yml')
myenv = Environment.from_conda_specification(name="myenv", file_path="myenv.yml")
inference_config = InferenceConfig(entry_script="inference/score.py", environment=myenv)

In [70]:
%%time
aks_service_name ='aks-service'

aks_service = Model.deploy(workspace=ws,
                           name=aks_service_name,
                           models=[model],
                           inference_config=inference_config,
                           deployment_config=aks_config,
                           deployment_target=aks_target)

aks_service.wait_for_deployment(show_output = True)
print(aks_service.state)

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2022-02-04 00:03:38+05:30 Creating Container Registry if not exists.
2022-02-04 00:03:38+05:30 Registering the environment.
2022-02-04 00:03:40+05:30 Use the existing image.
2022-02-04 00:03:42+05:30 Creating resources in AKS.
2022-02-04 00:03:43+05:30 Submitting deployment to compute.
2022-02-04 00:03:43+05:30 Checking the status of deployment aks-service..
2022-02-04 00:07:02+05:30 Checking the status of inference endpoint aks-service.
Succeeded
AKS service creation operation finished, operation "Succeeded"
Healthy
Wall time: 4min 3s


In [71]:
aks_service.scoring_uri

'http://138.91.136.177:80/api/v1/service/aks-service/score'